# Chapter7
## Model Performance and Validation

In [4]:
import os, sys, time
import numpy as np
from math import floor, ceil
from matplotlib import pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from matplotlib.patches import Patch, Rectangle
from matplotlib.lines import Line2D
from tqdm import tqdm
import h5py
# import python library
sys.path.append(os.path.join(os.getcwd().split(os.environ.get('USER'))[0],os.environ.get('USER'), 'wdml', 'py'))
from sample import Sample
from dataset import Dataset
from database import Database
from dataset_simulation import DatasetSimulation
from cfar_detector import CFARDetector
from nn_detector import NeuralNetworkDetector
from yolo_detector import YOLODetector

dataset_loc = os.path.join(os.getcwd().split(os.environ.get('USER'))[0],os.environ.get('USER'), 'wdml', 'data','datasets', 'awdEvents1')
database_loc = os.path.join(os.getcwd().split(os.environ.get('USER'))[0],os.environ.get('USER'), 'wdml', 'data','databases', 'awdEvents1')
result_loc = os.path.join(os.getcwd().split(os.environ.get('USER'))[0],os.environ.get('USER'), 'wdml', 'data','results', 'awdEvents1')
dataset_sim_loc = os.path.join(os.getcwd().split(os.environ.get('USER'))[0],os.environ.get('USER'), 'wdml', 'data','datasets', 'simulations', 'simple' ,'whistler')
site = 'marion'
database = Database(dataset_loc, database_loc, site)
files = database.get_train()

linewidth = 2
font_title = 32
font_legend = 20
font_label = 24
font_ticks = 18
font = [font_title, font_legend, font_label, font_ticks, linewidth] # [None]*5

### 7.4 Model Comparison

In [5]:
# CCWK
transforms, transforms_params = ['slice','zscore','scale'], [[1.5,9.5],[None],[0,1]]
N,G,k,Ts,Tl,X_dB = 12,10,13,5,3,0.5
pfa = (1/(1+((10**(X_dB/10))/(2*N))))**(2*N)
kernel = CFARDetector(dataset_loc, site, files[0]).load_kernel('sim', [0.35,80,1])
results = []
for i in range(5):
    start = time.time()
    sample_duration = 0
    files = database.get_train()[150*i:150*i+50]
    for file in tqdm(files):
        sample = CFARDetector(dataset_loc, site, file)
        sample_duration += sample.get_time()[-1]
        sample.detection_bounding_boxes(transforms,transforms_params, kernel, 'fusion_cfar', [N,G,k,Ts,Tl,pfa], threshold=0, time_err=1, duration=True)
    end = time.time()
    process_duration = end-start
    results.append([sample_duration, process_duration])
results = np.array(results)
ratios = results[:,1]/results[:,0]
print(ratios.mean(), ratios.std())

100%|██████████| 50/50 [00:43<00:00,  1.15it/s]

0.17878062890283392 0.016535292336003843


In [7]:
#SDCNN
sample = Sample(dataset_loc, site, file)
transforms, transforms_params = 'zscore', None
name = 'cnn-3conv-2dense-4dropout-100epochs'
nn = NeuralNetworkDetector(dataset_loc, database_loc, site, transforms, transforms_params)
model = nn.load_model(name=name)
scaler = nn.load_scaler(name=name)
results = []
for i in range(5):
    start = time.time()
    sample_duration = 0
    files = database.get_train()[150*i:150*i+50]
    for file in tqdm(files):
        sample = CFARDetector(dataset_loc, site, file)
        sample_duration += sample.get_time()[-1]
        nn.detection_bounding_boxes(file,thresh=0.8, concav=True, concav_window=0.02,concav_error=-0.0, time_error=1)
    end = time.time()
    process_duration = end-start
    results.append([sample_duration, process_duration])
results = np.array(results)
ratios = results[:,1]/results[:,0]
print(ratios.mean(), ratios.std())

100%|██████████| 50/50 [01:32<00:00,  1.85s/it]

0.37629323515071206 0.006583782812161268


In [5]:
# YOLO
yolo = YOLODetector()
model_data_type = "real"
model_base = 'yolov3-tiny'
transforms, transforms_params = 'detrend', 'linear'
model_name = 'real_transfer_learning_detrend_linear'
yolo.load_model(model_base, model_data_type, model_name)
results = []
for i in range(5):
    start = time.time()
    sample_duration = 0
    files = database.get_train()[150*i:150*i+50]
    for file in tqdm(files):
        sample = CFARDetector(dataset_loc, site, file)
        sample_duration += sample.get_time()[-1]
        yolo.detection_bounding_boxes(dataset_loc, site, file, transforms, transforms_params,thresh=.8)
    end = time.time()
    process_duration = end-start
    results.append([sample_duration, process_duration])
results = np.array(results)
ratios = results[:,1]/results[:,0]
print(ratios.mean(), ratios.std())

100%|██████████| 50/50 [00:02<00:00, 21.11it/s]

0.009705474877542968 0.0010280131464050551
